In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, num_classes):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Initialize the model
input_size = 200 * 200 * 3  # Assuming images of size 200x200 with 3 channels
num_classes = 2  # Update based on your dataset
mlp_model = MLP(input_size=input_size, num_classes=num_classes)

# Print the model summary
print(mlp_model)

# Count and print the number of model parameters
num_params = sum(p.numel() for p in mlp_model.parameters())
print(f"Number of model parameters: {num_params}")

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)

# Transformations
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
])

# Load dataset (replace with your dataset path)
train_data = datasets.ImageFolder(r'C:\Users\sahuy\Downloads\Dataset\Dataset\train', transform=transform)
test_data = datasets.ImageFolder(r'C:\Users\sahuy\Downloads\Dataset\Dataset\test', transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Training function
def train(model, train_loader, test_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()

        running_loss = 0.0
        correct_train = 0
        total_train = 0

        # Training loop
        for images, labels in train_loader:
            images = images.view(images.size(0), -1)  # Flatten images

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Accumulate loss and accuracy
            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)

        # Calculate epoch metrics
        train_loss = running_loss / len(train_loader.dataset)
        train_accuracy = correct_train / total_train

        # Testing loop
        model.eval()
        correct_test = 0
        total_test = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images = images.view(images.size(0), -1)

                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                correct_test += (preds == labels).sum().item()
                total_test += labels.size(0)

        test_accuracy = correct_test / total_test
        end_time = time.time()

        # Print epoch summary
        print(f"Epoch [{epoch+1}/{num_epochs}], Time: {end_time - start_time:.2f}s, "
              f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
              f"Test Accuracy: {test_accuracy:.4f}")

# Start training
start_time = time.time()
train(mlp_model, train_loader, test_loader, criterion, optimizer, num_epochs=10)
end_time = time.time()

# Print total training time
print(f"Total Training Time: {end_time - start_time:.2f} seconds")


MLP(
  (model): Sequential(
    (0): Linear(in_features=120000, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=1024, out_features=1024, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=1024, out_features=512, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=512, out_features=256, bias=True)
    (10): ReLU()
    (11): Linear(in_features=256, out_features=2, bias=True)
  )
)
Number of model parameters: 124587266
Epoch [1/10], Time: 81.66s, Train Loss: 4.9197, Train Accuracy: 0.5500, Test Accuracy: 0.5000
Epoch [2/10], Time: 71.06s, Train Loss: 2.4928, Train Accuracy: 0.4813, Test Accuracy: 0.5000
Epoch [3/10], Time: 70.52s, Train Loss: 1.6171, Train Accuracy: 0.5250, Test Accuracy: 0.5000
Epoch [4/10], Time: 78.50s, Train Loss: 1.3951, Train Accuracy: 0.5188, Test Accuracy: 0.5000
Epoch [5/10], Time: 74.42s, Train Loss: 1.2175, Train 